In [1]:
import sys
sys.path.append('../analysis')
from utils.all import *

DEBUG:matplotlib data path: C:\Users\heine\AppData\Roaming\Python\Python39\site-packages\matplotlib\mpl-data
DEBUG:CONFIGDIR=C:\Users\heine\.matplotlib
DEBUG:interactive is False
DEBUG:platform is win32
DEBUG:CACHEDIR=C:\Users\heine\.matplotlib
DEBUG:Using fontManager instance from C:\Users\heine\.matplotlib\fontlist-v330.json


In [3]:
data = load_data('inspection_annotated')

INFO:Loading files: ['inspection_annotated/batch_1_ayush.json', 'inspection_annotated/batch_1_rachel.json', 'inspection_annotated/batch_1_vinayak.json', 'inspection_annotated/batch_1_vishnesh.json', 'inspection_annotated/batch_2_ayush.json', 'inspection_annotated/batch_2_rachel.json', 'inspection_annotated/batch_2_vinayak.json', 'inspection_annotated/batch_2_vishnesh.json']

INFO:Found users: {'rachel', 'ayush', 'vinayak', 'vishnesh'}



In [12]:
# Deletes any annotations in the adjudicated data
del_anno = {
    'deletion': {},
    'substitution': {},
    'insertion': {},
    'split': {},
    'reorder': {},
    'structure': {}
}

for sent in data:
    sent['annotations'] = del_anno

with open(f"adjudicated_span_selection.json", "w") as f:
   json.dump(data, f, indent=4)

In [26]:
# Assigns each sentence to all users who did not adjudicate the sentence
users = list(set([x['user'] for x in data]))
assignment = {}
for user in users:
    assignment[user] = []
for sent in data:
    for user in [u for u in users if u != sent['user']]:
        assignment[user] += [sent]

for u in users:
    print(f'{u}: {len(assignment[u])}')

rachel: 528
ayush: 513
vinayak: 530
vishnesh: 529


In [27]:
# Saves rating task
batch_size = 140

for user, sents in assignment.items():
    for i in range(math.ceil(len(sents) / batch_size)):
        end_idx = (i+1)*batch_size
        if end_idx >= len(sents):
            end_idx = len(sents)

        path = f"inspection_rating/{user}"
        if not os.path.exists(path):
            os.makedirs(path)

        with open(f"{path}/batch_{i+1}.json", "w") as f:
            json.dump(sents[i*batch_size:end_idx], f, indent=4)